In [1]:
import sys
# To avoid ImportError: attempted relative import with no known parent package
sys.path.append("/Users/jocareher/Library/CloudStorage/OneDrive-Personal/Educación/PhD_UPF_2023/Face_Detection/")
sys.path.append("/Users/jocareher/Library/CloudStorage/OneDrive-Personal/Educación/PhD_UPF_2023/Face_Detection/scripts")

from scripts import data_setup, utils, parser, train, engine, visualization
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2 import model_zoo

In [2]:
# Setting random seed for reproducibility
utils.set_seed()

# Set the root directory
root_dir = "/Users/jocareher/Downloads/face_dataset/"

# Define the class labels
class_labels = ["3/4_left_sideview","3/4_rigth_sideview","Frontal","Left_sideview","Right_sideview"]

# Register the dataset for training and get datasets sizes
train_size, val_size, test_size = data_setup.register_datasets(root_dir=root_dir, class_labels=class_labels)

# DATALOADER config
train_dataset = "train"
val_dataset = "val"
num_workers = 2

# Model config
base_config_path = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
rotated_bbox_config_path = "../configs/rotated_bbox_config.yaml"
pretrained_model_url = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
num_classes = len(class_labels)
device = "cpu"
unfreeze_backbone = False
freeze_at_block = 0
output_dir = "/Users/jocareher/Downloads/output_fe/"

# Training config
ims_per_batch = 2
checkpoint_period = 1000
base_lr = 0.0025
epochs = 100
batch_size_per_image = 128
warm_steps = 1000
gamma = 0.1

# Set the model and training configuration
model_and_train_config = engine.setup_cfg(
    base_config_path=base_config_path,
    rotated_bbox_config_path=rotated_bbox_config_path,
    pretrained_model_url=pretrained_model_url,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    output_dir=output_dir,
    num_classes=num_classes,
    num_workers=num_workers,
    device=device,
    unfreeze_backbone=unfreeze_backbone,
    freeze_at_block=freeze_at_block,
    ims_per_batch=ims_per_batch,
    checkpoint_period=checkpoint_period,
    base_lr=base_lr,
    epochs=epochs,
    train_data_size=train_size,
    batch_size_per_image=batch_size_per_image,
    warm_steps=warm_steps,
    gamma=gamma,
)


Config '../configs/rotated_bbox_config.yaml' has no VERSION. Assuming it to be compatible with latest v2.


In [3]:
trainer = train.FaceTrainer(cfg=model_and_train_config)

[02/01 11:13:24 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

In [ ]:
# Re-initialize the configuration for evaluation
cfg = get_cfg()

# Load the configuration for a pre-trained model
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.merge_from_file("../configs/rotated_bbox_config.yaml")

# Load the trained model weights
cfg.MODEL.WEIGHTS = "/Users/jocareher/Downloads/model_0004999.pth"

# Set the device to CUDA (GPU support)
cfg.MODEL.DEVICE = 'cpu' # cpu

# Set the number of classes
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5

# Set the threshold for object detection
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("val",)
# Initialize the predictor with the configuration
predictor = DefaultPredictor(cfg)

# Initialize the evaluator
evaluator = train.FaceTrainer.build_evaluator(cfg, "val", output_folder="/Users/jocareher/Downloads/")

# Build the validation data loader
val_loader = build_detection_test_loader(cfg, "val")

# Perform evaluation on the validation dataset
inference_on_dataset(trainer.model, val_loader, evaluator)

In [ ]:
visualization.visualize_predictions(
    cfg_path="COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml",
    weights_path="/Users/jocareher/Downloads/model_final.pth",
    dataset_name="val",
    num_classes=5,
    score_thresh=0.75,
    num_images=12,
    device='cpu'
)

In [17]:
import matplotlib.pyplot as plt
import cv2
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultPredictor

def visualize_all_predictions(
    cfg_path: str,
    weights_path: str,
    dataset_name: str,
    num_classes: int,
    score_thresh: float,
    device: str = "cpu",
) -> None:
    """
    Visualizes predictions on all images from a specified dataset using a Detectron2 model.

    Args:
        cfg_path (str): Path to the configuration file for the model.
        weights_path (str): Path to the trained model weights.
        dataset_name (str): Name of the dataset to visualize (e.g., "val", "test").
        num_classes (int): Number of classes for the model.
        score_thresh (float): Score threshold for making predictions.
        device (str): Computation device to use ('cpu' or 'cuda').

    Displays each image with model predictions including bounding boxes, labels, and confidence scores.
    """
    # Initialize Detectron2 configuration
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(cfg_path))
    cfg.merge_from_file("../configs/rotated_bbox_config.yaml")
    cfg.MODEL.WEIGHTS = weights_path
    cfg.MODEL.DEVICE = device
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = score_thresh
    cfg.DATASETS.TEST = (dataset_name,)
    predictor = DefaultPredictor(cfg)

    # Retrieve dataset dictionaries from the registered dataset
    dataset_dicts = DatasetCatalog.get(dataset_name)

    # Visualize each image individually
    for d in dataset_dicts:
        img = cv2.imread(d["file_name"])
        outputs = predictor(img)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        v = Visualizer(
            img_rgb,
            metadata=MetadataCatalog.get(dataset_name).set(thing_classes=[
                "3/4_left_sideview", "3/4_rigth_sideview", "Frontal", "Left_sideview", "Right_sideview"
            ]),
            scale=0.5,
        )
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        
        plt.figure(figsize=(12, 8))
        plt.imshow(out.get_image())
        plt.axis("off")
        plt.show()

In [ ]:
iter, loss = data_setup.extract_losses("/Users/jocareher/Downloads/detectron_pw480719.jreyes.out")
visualization.plot_losses(iter, loss)

In [ ]:
visualize_all_predictions(
    cfg_path="COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml",
    weights_path="/Users/jocareher/Downloads/model_final.pth",
    dataset_name="val",
    num_classes=5,
    score_thresh=0.7,
    device='cpu'
)